In [1]:
import numpy as np
import pandas as pd
import json
import wikitextparser as wtp
import re
from collections import Counter
import difflib

/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
flatten = lambda l: [item for sublist in l for item in sublist if len(item) is not 0]

In [221]:
train = pd.read_csv("../data/train_Youto_sentence.csv", dtype={'_id': str})
train.head()

,_id,sentence,label
0,1385415,アビエチン酸（Abietic acid）は松脂に含まれる主要な刺激性成分で、ロジンを構成する...,0
1,3430792,構造的には重篤な造血機能障害を生ずるアミノピリンと同系統であるが、発癌作用のあるニトロソアミ...,0
2,3430792,心房性の徐脈を伴う急性下壁心筋梗塞（コーニス症候群（英語版））が報告されている。,0
3,3430792,医療用医薬品の添付文書に記載されている重大な副作用は、ショック、皮膚粘膜眼症候群（Steve...,0
4,3430792,重篤な副作用により、いくつかの国では製造販売が禁止されたが、日本やイタリア、ドイツ、スペイン...,0


In [4]:
def read_jasonl(filename):
    with open(filename) as f:
        return [json.loads(line.rstrip('\r\n')) for line in f.readlines()]

In [199]:
wiki_data = read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

## サブカテゴリ名を取得

In [193]:
heading_list = Counter(flatten([p['heading'] for p in dump_data]))

In [209]:
pd.DataFrame.from_dict(
    heading_list, orient='index', columns=['count_']
).reset_index().rename(columns={'index': 'head'}).sort_values('count_', ascending=False).to_csv("../data/head_name_list.csv", index=False)

## サブタイトル名を学習データに追加

In [9]:
def sub(text, s):
    for i in s:
        if isinstance(i, str):
            text = text.replace(i, '')
        else:
            text = text.replace(i.string, '')
    
    return text

In [10]:
def clean_text(source_text):
    clean_text = sub(source_text.contents, source_text.templates)
    clean_text = sub(clean_text, source_text.tags())
    clean_text = sub(clean_text, source_text.external_links)

    clean_text = re.sub(r'\n|\t|\r', '', clean_text)
    clean_text = re.sub(r'={2,}.*?={2,}', '', clean_text)
    clean_text = re.sub(r'\[\[[^\]]+:.+?\]\]', '', clean_text)
    clean_text = re.sub(r'\[\[[^\]]+?\||\]\]|\[\[', '', clean_text)
    clean_text = re.sub(r'\'{2,}|\*+|#+', '', clean_text)
    clean_text = re.sub(r'<[^>]*?>.*?<\/[^>]*?>', '', clean_text)
    clean_text = re.sub(r'{{.*?}}|{.*?}', '', clean_text)
    
    return clean_text

In [285]:
train = train.assign(heading = '')

new_train_df = pd.DataFrame()

for _id in train._id.unique():
    train_article = train.loc[train._id == _id].reset_index(drop=True)
    
    article = [entry for entry in wiki_data if entry['index']['_id'] == _id][0]
    parsed = wtp.parse(article['source_text'])
    
    for source in parsed.sections[1:]:
        m = re.search(r'==+\s+?([^=]+?)\s+?==+', source.string)
        if m:
            heading = m.group(1)
        else:
            heading = ''

        section_text = clean_text(source)
        for s in re.findall(r'.*?。', section_text):
            m_sentence = difflib.get_close_matches(s, train_article.sentence.values, n=1)
            if len(m_sentence) > 0 and len(heading) > 0:
                train_article.loc[train_article.sentence == m_sentence[0], ['heading']] = \
                    train_article.loc[train_article.sentence == m_sentence[0], ['heading']] + ' ' + heading
                #train_article.loc[train_article.sentence == m_sentence[0], 'heading'] = heading

    if len(train_article.loc[train_article.heading != '']) is 0:
        train_article.loc[train_article.heading == '', ['heading']] = 'NO_SUBTITLE'
    else :
        train_article.loc[0:train_article[train_article.heading != ''].index[0], ['heading']] = 'NO_SUBTITLE'
        
    while len(train_article.loc[train_article.heading == '']) > 0: 
        train_article.loc[train_article.heading == '', ['heading']] = \
            train_article.loc[train_article.loc[train_article.heading == '', ['heading']].index - 1, 'heading'].values[0]
        
    new_train_df = new_train_df.append(train_article)

In [286]:
new_train_df.to_csv("../data/train_Youto_sentence_and_heading.csv", index=False)